#### ADS 509
#### Assignment 2
#### Abby Tan
#### GitHub Link: https://github.com/Abby-Tan/ADS509_Assignment_4

## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
import pandas as pd

In [3]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [4]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            select * from conventions
                            ''')

for row in query_results :
    # store the results in convention_data
    convention_data.append(row)
    pass # remove this

In [5]:
pd.DataFrame(convention_data).head(3)

,0,1,2,3,4,5,6,7
0,Democratic,4,Unknown,1,00:00,Skip to content The Company Careers Press Free...,127,www_rev_com_blog_transcripts2020-democratic-na...
1,Democratic,4,Speaker 1,1,00:33,I’m here by calling the full session of the 48...,41,www_rev_com_blog_transcripts2020-democratic-na...
2,Democratic,4,Speaker 2,1,00:59,"Every four years, we come together to reaffirm...",17,www_rev_com_blog_transcripts2020-democratic-na...


In [6]:
convention_data = [[col[5],col[0]] for col in convention_data]

In [7]:
pd.DataFrame(convention_data).head()

,0,1
0,Skip to content The Company Careers Press Free...,Democratic
1,I’m here by calling the full session of the 48...,Democratic
2,"Every four years, we come together to reaffirm...",Democratic
3,We fight for a more perfect union because we a...,Democratic
4,"We must come together to defeat Donald Trump, ...",Democratic


Let's look at some random entries and see if they look right. 

In [8]:
random.choices(convention_data,k=10)

[['American people are in her heart. I know that she fight like [inaudible 02:10:00]. Dr.',
  'Democratic'],
 ['Foreign and domestic.', 'Republican'],
 ['He’s always eating ice cream. Usually it’s vanilla, with chocolate?',
  'Democratic'],
 ['Not Episcopal Church, I found that there’s that famous phrase from Kierkegaard, faith sees best in the dark. I find the one thing it gives me, and I’m not trying to process. I’m not trying to convince you to share my religious views. But for me, it’s important because it gives me some reason to have hope and purpose.',
  'Democratic'],
 ['Utah.', 'Democratic'],
 ['Yes.', 'Democratic'],
 ['At home, the president is the clear choice on jobs in the economy. He’s moved America forward while Joe Biden has held America back. When Joe was VP, I was governor of the great state of South Carolina. We had a pretty good run. Manufacturers of all kinds flocked to our state from overseas, creating tens of thousands of American jobs. People were referring to So

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [9]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2891 as features in the model.


In [72]:
word_list = []
word_bool = []

def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    word_list = [word for word in text.lower().split()]
    word_list = [word for word in Counter(word_list)]
    for word in word_list:
        if word in fw:
            word_bool.append(True)
        else:
             word_bool.append(False)
    ret_dict = dict(zip(word_list,word_bool))
    return(ret_dict)  

In [73]:
assert(len(feature_words)>0)

In [74]:
conv_features("donald is the president",feature_words)

{'donald': False, 'is': True, 'the': True, 'president': True}

In [75]:
assert(conv_features("donald is the president",feature_words)==
       {'donald':False,
        'is':True,
        'the':True,
        'president':True})

In [76]:
conv_features("people are american in america",feature_words)

{'people': False, 'are': True, 'american': True, 'in': True, 'america': False}

In [79]:
assert(conv_features("people are american in america",feature_words)==
                    {'people': False,
                     'are': True,
                     'american': True,
                     'in': True,
                     'america': False})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [62]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [63]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [64]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.454


In [65]:
classifier.show_most_informative_features(25)

Most Informative Features
                    land = True           Republ : Democr =     29.6 : 1.0
                   media = True           Republ : Democr =     22.3 : 1.0
             enforcement = True           Republ : Democr =     21.3 : 1.0
                   trade = True           Republ : Democr =     20.2 : 1.0
                  school = False          Republ : Democr =     19.2 : 1.0
                freedom, = True           Republ : Democr =     18.2 : 1.0
                      j. = True           Republ : Democr =     18.2 : 1.0
                 destroy = True           Republ : Democr =     14.0 : 1.0
                   votes = True           Democr : Republ =     13.5 : 1.0
                    kept = True           Republ : Democr =     13.0 : 1.0
               delivered = True           Republ : Democr =     11.9 : 1.0
                  first. = True           Republ : Democr =     11.9 : 1.0
                policies = True           Republ : Democr =     11.9 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

_Your observations to come._

In the classifier, the top 25 informative features are mostly from the Republican. Not too suprise that the words like media, trade, freedom, etc. are related to Republican, but it's interesting that the word of trump is in the top 25 informative features.


## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [17]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [18]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [19]:
pd.DataFrame(results).head(3)

,0,1,2
0,Mo Brooks,Republican,"b'""Brooks Joins Alabama Delegation in Voting A..."
1,Mo Brooks,Republican,"b'""Brooks: Senate Democrats Allowing President..."
2,Mo Brooks,Republican,"b'""NASA on the Square"" event this Sat. 11AM \x..."


In [20]:
tweet_data = [[col[2],col[1]] for col in results]

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

In [21]:
pd.DataFrame(tweet_data).head(3)

,0,1
0,"b'""Brooks Joins Alabama Delegation in Voting A...",Republican
1,"b'""Brooks: Senate Democrats Allowing President...",Republican
2,"b'""NASA on the Square"" event this Sat. 11AM \x...",Republican


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [22]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [23]:

for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet,feature_words)) #'Gotta fill this in'
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: b'Earlier today, I spoke on the House Floor abt protecting health care for women and praised @PPmarmonte for their work on the Central Coast. https://t.co/WqgTRzT7VV'
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: b'Go Tribe! #RallyTogether https://t.co/0NXutFL9L5'
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: b"Apparently, Trump thinks it's just too easy for students overwhelmed by the crushing burden of debt to pay off student loans #TrumpBudget https://t.co/ckYQO5T0Qh"
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: b'We\xe2\x80\x99re grateful for our first responders, our rescue personnel, our firefighters, our police, and volunteers who have been working tirelessly to keep people safe, provide much-needed help, while putting their own lives on the line.\n\nhttps://t.co/eZPv0vMIz3'
Actual party is Republican and our classife

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [69]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet,feature_words)) #"Gotta fill this in"
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [70]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3931, 'Democratic': 391}),
             'Democratic': defaultdict(int,
                         {'Republican': 5158, 'Democratic': 522})})

### Reflections

_Write a little about what you see in the results_ 

As the result shown, the prediction for Republican class has 3931 outputs that are predicted correctly, while 391 are misclassified. The model has good preformance on the Republican class. However, for Democraftic, there are 5158 outputs are predicted incorrect and only 522 is correct. Therefore, the model is not perforamnce well on the Democrafic class. 

---